In [2]:
from PyQt5.QtWidgets import QMessageBox
import numpy as np
import os
import cv2
from tqdm import tqdm
import pickle 

class train_test_data():
    TRAIN_DATADIR = "Images\Train"
    Validation_DATADIR = "Images\Validation"
    CATEGORIES = ["angry","disgust","fear" ,"happy","sad","surprise"]
    IMG_SIZE = 48
    training_data = []
    Validation_data=[]
    X = [] #for Input features of Train Data
    y = [] #for Labels of Train Data
    VAl_X = [] #for Input features of Validation_data
    VAl_y = [] #for Labels of Validation_data
    
#-----------Split features and labels from training and validation dataset also ------------------------
    def split_train_text_data(self):
        #----------Store traning and validating dataset features and labels into different variables--------
        for features,label in self.training_data:
            self.X.append(features)
            self.y.append(label)

        for features,label in self.Validation_data:
            self.VAl_X.append(features)
            self.VAl_y.append(label)

        # reshape data to have a single channel
        self.y=np.array(self.y)
        self.VAl_y=np.array(self.VAl_y)

 #----------------Shape the dataset according to your model requirment(input)--------------
        self.X=np.array(self.X)
        self.VAl_X = np.array(self.VAl_X)
        self.X = self.X.reshape((self.X.shape[0], self.X.shape[1], self.X.shape[2], 1))
        self.VAl_X = self.VAl_X.reshape((self.VAl_X.shape[0], self.VAl_X.shape[1], self.VAl_X.shape[2], 1))
        self.X = self.X.astype('float32') / 255.0
        self.VAl_X = self.VAl_X.astype('float32') / 255.0
        


#------store Encoded Data Set for Futhare use if required Shape the dataset according to your model requirment(input)--

    def save_DataSet(self):
        pickle_out = open("SavedDataSet\X.pickle","wb")
        pickle.dump(self.X, pickle_out)
        pickle_out.close()

        pickle_out = open("SavedDataSet\y.pickle","wb")
        pickle.dump(self.y, pickle_out)
        pickle_out.close()

        pickle_out = open("SavedDataSet\VAl_X.pickle","wb")
        pickle.dump(self.VAl_X, pickle_out)
        pickle_out.close()

        pickle_out = open("SavedDataSet\VAl_y.pickle","wb")
        pickle.dump(self.VAl_y, pickle_out)
        pickle_out.close()

        
        
#---------Create the training Data According to our dataset inputs and labels----------------------
    def create_training_data(self):  #Function That Create Training Data Form File
        for category in self.CATEGORIES:  # do for all Face Categories
            path = os.path.join(self.TRAIN_DATADIR,category)  #create path to Faces
            class_num = self.CATEGORIES.index(category)  #get the classification(0 to 5). 0=Angry 1=Disgust....

            for img in tqdm(os.listdir(path)):  # iterate over each image per Face Categories
                try:
                    Original_img = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                   
                    laplacian =cv2.Laplacian(Original_img,cv2.CV_64F)
                    img_array=Original_img-laplacian 

                    self.training_data.append([img_array, class_num])  # add this to our training_data
                except Exception as e:  # in the interest in keeping the output clean...
                    pass

#---------Create the validation Data According to our dataset inputs and labels----------------------
    def create_Validation_data(self):
        for category in self.CATEGORIES:  # do for all Face Categories
            path = os.path.join(self.Validation_DATADIR,category)  #create path to Faces
            class_num = self.CATEGORIES.index(category)  #get the classification(0 to 5).0=Angry 1=Disgust ....

            for img in tqdm(os.listdir(path)):  # iterate over each image per Face Categories
                try:
                    Original_img = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to array
                    laplacian =cv2.Laplacian(Original_img,cv2.CV_64F)
                    img_array=Original_img-laplacian

                    self.Validation_data.append([img_array, class_num])  # add this to our Validation_data
                except Exception as e:  # in the interest in keeping the output clean...
                    pass
                
#def plotImages(images_arr):
 #   fig, axes = plt.subplots(1, 5, figsize=(20,20))
  #  axes = axes.flatten()
   # for img, ax in zip( images_arr, axes):
    #    ax.imshow(img)
     #   ax.axis('off')
        
   # plt.tight_layout()
   # plt.show()
#aug = [training_data[0][0] for i in range(10)]
#plotImages(aug)

    def update_Train_Test_Data(self):
       
        self.create_training_data()
        print("creation of training data Done.....")
        print('lenght of training data : %d' %len(self.training_data))    
        print(" ")

        self.create_Validation_data()
        print("creation of Validation data Done.....")
        print('lenght of validation data : %d' %len(self.Validation_data))

        print(" ")
        print("Spliting training and validation dataset into features and labels...")
        self.split_train_text_data()
        print("training and validation dataset splited successfully..")

        print("saving data set to local file...")
        self.save_DataSet()
        print("Dataset saved succesfully..")
        
        msgBox = QMessageBox()
        msgBox.setIcon(QMessageBox.Information)
        msgBox.setText("Update Dataset succesfully")
        msgBox.setWindowTitle("Update Dataset")
        msgBox.setStandardButtons(QMessageBox.Ok)
        
        returnValue = msgBox.exec()